As empresas listadas na B3 são claramente empresas ativas, operantes de fato. Obter a lista dos CNPJs dessas empresas nos permitirá rotular algumas empresas da base nacional como existentes de fato.

Vamos obter os CNPJs das empresas listadas na B3 em 2021 a partir de um site de informações sobre investimentos: www.infomoney.com.br

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Obtém o html da página web que possui os CNPJs de todas as empresas listadas na B3 em 2021:
response = requests.get('https://www.infomoney.com.br/minhas-financas/confira-o-cnpj-das-acoes-negociadas-em-bolsa-e-saiba-como-declarar-no-imposto-de-renda/')
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# Obtém a tabela html com os cnpjs:
tabela_html = soup.find_all("div", {"class": "container-article-table"})

# Obtém as linhas da tabela
linhas_html = tabela_html[0].find_all('tr')

cnpjs = list()

# Para cada linha, pega apenas o CNPJ:
for linha_html in linhas_html:
    cnpjs.append(linha_html.findAll(lambda tag: tag.name=='td')[3].getText())

# Remove a linha que corresponde ao título
cnpjs.remove('CNPJ')

# Cria o dataframe com a lista de CNPJs:
df = pd.DataFrame(cnpjs, columns=["CNPJ"])

# Faz limpeza dos valores
df.CNPJ = df.CNPJ.str.replace('[\.\-\/]', '', regex=True)

print(df)

                CNPJ
0    012091809000155
1    001851771000155
2    012528708000107
3    037663076000107
4    004128563000110
..               ...
445  059105999000186
446  005721735000128
447  042278473000103
448  033228024000151
449  008807432000110

[450 rows x 1 columns]


Há uma impropriedade nos CNPJs listados: todos possuem um zero a mais no início:

In [4]:
df.CNPJ.str[0].value_counts()

0    450
Name: CNPJ, dtype: int64

In [6]:
# Removendo o zero a mais:
df.CNPJ = df.CNPJ.str[1:]

In [7]:
# Verificando resultado:
print(df.describe)
print(df.CNPJ.nunique())

<bound method NDFrame.describe of                CNPJ
0    12091809000155
1    01851771000155
2    12528708000107
3    37663076000107
4    04128563000110
..              ...
445  59105999000186
446  05721735000128
447  42278473000103
448  33228024000151
449  08807432000110

[450 rows x 1 columns]>
450


In [8]:
# Salvando resultado:
df.to_csv('C:\DATASETS\Empresas Existentes de Fato\empresas_listadas_B3.csv', index=False)